## [NASA logs for July 1995](http://ita.ee.lbl.gov/html/contrib/NASA-HTTP.html)

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

In [1]:
!wget ftp://ita.ee.lbl.gov/traces/NASA_access_log_Jul95.gz
!gunzip NASA_access_log_Jul95.gz

--2019-03-16 16:24:50--  ftp://ita.ee.lbl.gov/traces/NASA_access_log_Jul95.gz
           => ‘NASA_access_log_Jul95.gz’
Resolving ita.ee.lbl.gov (ita.ee.lbl.gov)... 131.243.2.164, 2620:83:8000:102::a4
Connecting to ita.ee.lbl.gov (ita.ee.lbl.gov)|131.243.2.164|:21... connected.
Logging in as anonymous ... Logged in!
==> SYST ... done.    ==> PWD ... done.
==> TYPE I ... done.  ==> CWD (1) /traces ... done.
==> SIZE NASA_access_log_Jul95.gz ... 20676672
==> PASV ... done.    ==> RETR NASA_access_log_Jul95.gz ... done.
Length: 20676672 (20M) (unauthoritative)

NASA_access_log_Jul   3%[                    ] 772.19K   244KB/s    eta 79s    ^C
gzip: NASA_access_log_Jul95 already exists; do you wish to overwrite (y or n)? ^C


## Load logs into a RDD

In [3]:
sc = spark.sparkContext
# INSERT CODE HERE
rdd = sc.textFile('NASA_access_log_Jul95')
sample_line = rdd.take(1)[0]
print(sample_line)

199.72.81.55 - - [01/Jul/1995:00:00:01 -0400] "GET /history/apollo/ HTTP/1.0" 200 6245


In [ ]:
#weblog.findall(sample_line)[0]
weblog.findall("Hello world")

### How many non-empty lines?

In [50]:
# INSERT CODE HERE
def non_empty(line):
    return len(line) > 0

non_empty(sample_line)

rdd.filter(non_empty).count()

1891715

### Create a function to parse Apache logs and, at the same time, counts malformed lines

In [6]:
import re
# host unused user time method resource status size referer agent 
pattern = '^(\S+) \S+ (\S+) \[([\w:/]+\s[+\-]\d{4})\] "(\S+)\s?(\S+)?\s?(\S+)?" (\d{3}|-) (\d+|-)\s?"?([^"]*)"?\s?"?([^"]*)?"?$'
weblog = re.compile(pattern=pattern)

# TIP: you can use Accumulator to count lines
# Make sure you return an empty string ('') 
# whenever you find a malformed line

malformed_lines = sc.accumulator(0)
#bad_line = spark.sparkContext.accumulator(0)
def process_logs(line):
    # INSERT CODE HERE
    global malformed_lines
    if len(weblog.findall(line)) == 0:
        malformed_lines.add(1)
        return ''
    else:
        return weblog.findall(line)[0]

#def process_logs(line):
#    try:
#        return weblog.findall(line)[0]
#    except IndexError:
#        malformed_lines.add(1)
#        return ''

log_entries = rdd.map(process_logs)
log_entries.count() # just for triggering lazy spark
print(f"Malformed lines: {malformed_lines.value}")

Malformed lines: 1902


In [ ]:
# Taking one row
print(nasa.take(1)[0])

In [ ]:
# Applies parsing function and 
parsed_nasa = nasa.map(process_logs)
# Check how many empty lines you got
parsed_nasa.filter(lambda line: line == '').count()

In [ ]:
# Check the accumulator value
# Both values should match

### Create a function to convert a tuple into a Row object

In [ ]:
import time, datetime

def parse_time(str_time):
    tt = time.strptime(str_time[:-6], "%d/%b/%Y:%H:%M:%S")
    return datetime.datetime(*tt[:6])

In [ ]:
from pyspark.sql import Row

# The size of the response in the logs may be '-' sometimes
# Create a function that converts the size to an integer value
# and returns -1 for invalid sizes, while also keeping track
# of how many times it happened in the NO_SIZE variable
# (hint: Accumulator)

no_size = ...

def to_int(value):
    # INSERT CODE HERE
    pass

# Leveraging on the previous function, that converts a line
# from the logs into a tuple using RegEx, populate a Row object
# to make it ready to turn into a DataFrame

def log_to_row(line):
    # INSERT CODE HERE
    # Process the line into fields
    
    # If it is a malformed line, it will return an empty string
    # So we create mock fields for it
    if fields == '':
        fields = [''] * 10
        fields[2] = '01/Jan/1900:00:00:00 -0000'
        fields[6] = 0

    # INSERT CODE HERE
    # Create a row object with all the fields
    # host, user, time, method, resource, protocol, status, size, referer, agent
    row = ...
    
    return row

In [ ]:
log_to_row(nasa.take(1)[0])

### Create a DataFrame

In [ ]:
# Use the original RDD (nasa), apply your function to it
# and create a DataFrame
df = ...

In [ ]:
df.show()

In [ ]:
df.count()

In [ ]:
# How many lines without size?
no_size.value

In [ ]:
# How many lines without size?
# Both values should match
df.filter('size == -1').count()

### Which hosts were responsible for the largest amount of traffic?

In [ ]:
from pyspark.sql import functions as F

df.groupby('host').agg(F.sum('size').alias('total_size')).orderBy(F.desc('total_size')).show()

### Which resources were the most requested?

In [ ]:
df.groupby('resource').agg(F.count(F.lit(1)).alias('count')).orderBy(F.desc('count')).limit(20).toPandas()